# Alloy BTC-PAXG Trading Strategy Quick Start

This Jupyter Notebook is a self-contained quick-start guide for the Alloy BTC-PAXG trading strategy, designed as an alternative to the Streamlit application (`app.py`) for users who cannot run it locally. The strategy dynamically allocates between Bitcoin (BTC) and PAX Gold (PAXG) based on momentum and volatility to balance growth and stability.

## Purpose
- Provide an interactive interface for backtesting, generating trading signals, and optimizing strategy parameters.
- Offer comprehensive visualizations and metrics comparable to the Streamlit app.
- Serve as an educational tool for understanding the Alloy strategy.

## Table of Contents
1. [Introduction](#Introduction)
2. [Data Loading](#Data-Loading)
3. [Strategy Implementation](#Strategy-Implementation)
4. [Backtesting](#Backtesting)
5. [Results Visualization](#Results-Visualization)
6. [Parameter Experimentation](#Parameter-Experimentation)
7. [Trading Signals](#Trading-Signals)
8. [Strategy Optimization](#Strategy-Optimization)
9. [Next Steps](#Next-Steps)

## Prerequisites
Install the required packages:
```bash
pip install yfinance pandas numpy matplotlib plotly ipywidgets optuna
```
Ensure Jupyter supports widgets: `pip install ipywidgets` and `jupyter nbextension enable --py widgetsnbextension`.

## Disclaimer
**This code is for educational purposes only. Cryptocurrency investments are highly volatile and carry significant risks. Always conduct your own research and consult a qualified financial advisor before making investment decisions.**

## Data Loading

Download historical BTC and PAXG price data using `yfinance`. Select a date range for analysis, with error handling for API issues.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import logging
from datetime import datetime, timedelta
from ipywidgets import interact, DatePicker, Button, Output, VBox
from IPython.display import display, HTML

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def download_data(start_date: str, end_date: str, max_retries: int = 3) -> pd.DataFrame:
    """Download and prepare BTC and PAXG data with retry logic."""
    for attempt in range(max_retries):
        try:
            logger.info(f"Attempt {attempt + 1}/{max_retries} to download data")
            btc = yf.download('BTC-USD', start=start_date, end=end_date, progress=False)
            paxg = yf.download('PAXG-USD', start=start_date, end=end_date, progress=False)

            common_dates = btc.index.intersection(paxg.index)
            data = pd.DataFrame(index=common_dates)
            data['BTC'] = btc.loc[common_dates, 'Close']
            data['PAXG'] = paxg.loc[common_dates, 'Close']
            data = data.dropna()

            if data.empty:
                raise ValueError("No valid data retrieved")

            logger.info(f"Data loaded: {len(data)} days from {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
            return data

        except Exception as e:
            logger.error(f"Error on attempt {attempt + 1}: {str(e)}")
            if attempt < max_retries - 1:
                import time
                time.sleep(5 * (attempt + 1))
            else:
                logger.error("Failed to download data after multiple attempts")
                return None

# Interactive date selection
start_date_picker = DatePicker(description='Start Date', value=datetime(2020, 1, 1))
end_date_picker = DatePicker(description='End Date', value=datetime.now())
load_data_button = Button(description='Load Data')
data_output = Output()

historical_data = None

def on_load_data_clicked(b):
    global historical_data
    with data_output:
        data_output.clear_output()
        if start_date_picker.value and end_date_picker.value:
            start_date = start_date_picker.value.strftime('%Y-%m-%d')
            end_date = end_date_picker.value.strftime('%Y-%m-%d')
            historical_data = download_data(start_date, end_date)
            if historical_data is not None:
                display(HTML(f"<b>Data loaded successfully:</b> {len(historical_data)} days"))
                display(historical_data.tail())
            else:
                display(HTML("<b style='color:red'>Error: Failed to load data. Try a different date range.</b>"))
        else:
            display(HTML("<b style='color:red'>Error: Please select valid dates.</b>"))

load_data_button.on_click(on_load_data_clicked)
display(VBox([start_date_picker, end_date_picker, load_data_button, data_output]))

## Strategy Implementation

The Alloy strategy dynamically adjusts allocations between BTC and PAXG based on momentum and volatility. The `AlloyPortfolioBOME` class implements the advanced strategy with parameter optimization.

**Key Parameters**:
- `initial_capital`: Starting portfolio value (e.g., $10,000).
- `momentum_window`: Period for BTC momentum (e.g., 30 days).
- `volatility_window`: Period for volatility (e.g., 60 days).
- `momentum_threshold_bull/bear`: Thresholds for bullish/bearish signals (e.g., 10%/-5%).
- `max/min_btc_allocation`: Allocation bounds for BTC (e.g., 90%/20%).
- `rebalance_frequency`: Days between rebalancing (e.g., 3 days).
- `transaction_cost`: Trading fee (e.g., 0.1%).

In [ ]:
from typing import Dict, Tuple
import optuna

class AlloyPortfolioBOME:
    def __init__(self, initial_capital: float, momentum_window: int, volatility_window: int,
                 momentum_threshold_bull: float, momentum_threshold_bear: float,
                 max_btc_allocation: float, min_btc_allocation: float,
                 rebalance_frequency: int, transaction_cost: float, data: pd.DataFrame):
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.params = {
            'momentum_window': momentum_window,
            'volatility_window': volatility_window,
            'momentum_threshold_bull': momentum_threshold_bull,
            'momentum_threshold_bear': momentum_threshold_bear,
            'max_btc_allocation': max_btc_allocation,
            'min_btc_allocation': min_btc_allocation,
            'rebalance_frequency': rebalance_frequency
        }
        self.logger = logging.getLogger('AlloyBOME')
        self.btc_allocation = 0.5
        self.paxg_allocation = 0.5
        self.last_rebalance = None
        self.positions = {'BTC': 0, 'PAXG': 0}
        self.data = data

    def calculate_volatility(self, prices: pd.Series) -> pd.Series:
        returns = np.log(prices / prices.shift(1))
        volatility = returns.rolling(window=self.params['volatility_window']).std() * np.sqrt(252) * 100
        return volatility.bfill()

    def calculate_momentum(self, prices: pd.Series) -> pd.Series:
        return (prices / prices.shift(self.params['momentum_window']) - 1) * 100

    def get_market_context(self, btc_momentum: float, btc_volatility: float) -> str:
        momentum_state = 'bullish' if btc_momentum > self.params['momentum_threshold_bull'] else \
                         'bearish' if btc_momentum < self.params['momentum_threshold_bear'] else 'neutral'
        vol_state = 'low_vol' if btc_volatility < 30 else 'high_vol'
        return f"{momentum_state}_{vol_state}"

    def dynamic_rebalancing(self, historical_slice: pd.DataFrame, vol_btc: float, vol_paxg: float, current_price: pd.Series) -> bool:
        btc_momentum = self.calculate_momentum(historical_slice['BTC'])
        current_btc_momentum = btc_momentum.iloc[-1]
        context = self.get_market_context(current_btc_momentum, vol_btc)

        old_btc_alloc = self.btc_allocation
        if context.startswith('bullish'):
            self.btc_allocation = self.params['max_btc_allocation']
        elif context.startswith('bearish'):
            self.btc_allocation = self.params['min_btc_allocation']
        else:
            total_vol = vol_btc + vol_paxg
            self.btc_allocation = 0.5 if total_vol == 0 else max(self.params['min_btc_allocation'],
                                                                 min(self.params['max_btc_allocation'],
                                                                     1.2 - (vol_btc / total_vol)))
        self.paxg_allocation = 1 - self.btc_allocation

        should_rebalance = abs(self.btc_allocation - old_btc_alloc) > 0.05
        if self.last_rebalance is not None:
            days_since_last = (historical_slice.index[-1] - self.last_rebalance).days
            if days_since_last < self.params['rebalance_frequency']:
                self.btc_allocation = old_btc_alloc
                self.paxg_allocation = 1 - self.btc_allocation
                return False

        return should_rebalance

    def backtest(self, data: pd.DataFrame) -> pd.DataFrame:
        self.logger.info("Starting backtest...")
        vol_btc = self.calculate_volatility(data['BTC'])
        vol_paxg = self.calculate_volatility(data['PAXG'])

        portfolio_value = pd.Series(self.initial_capital, index=data.index, dtype=float)
        btc_alloc_series = pd.Series(0.5, index=data.index, dtype=float)
        paxg_alloc_series = pd.Series(0.5, index=data.index, dtype=float)
        transaction_fees = pd.Series(0.0, index=data.index, dtype=float)
        dca_value = pd.Series(self.initial_capital, index=data.index, dtype=float)

        self.positions['BTC'] = (self.initial_capital * 0.5) / data['BTC'].iloc[0]
        self.positions['PAXG'] = (self.initial_capital * 0.5) / data['PAXG'].iloc[0]
        self.last_rebalance = data.index[0]

        # DCA setup: Equal daily investments
        daily_investment = self.initial_capital / len(data)
        btc_dca_units = 0
        paxg_dca_units = 0

        for i, (date, row) in enumerate(data.iterrows()):
            current_vol_btc = vol_btc.get(date, 0)
            current_vol_paxg = vol_paxg.get(date, 0)
            historical_slice = data.loc[:date]

            # DCA strategy
            btc_dca_units += (daily_investment * 0.5) / row['BTC']
            paxg_dca_units += (daily_investment * 0.5) / row['PAXG']
            dca_value.iloc[i] = btc_dca_units * row['BTC'] + paxg_dca_units * row['PAXG']

            # Alloy strategy
            if (date - self.last_rebalance).days >= self.params['rebalance_frequency']:
                if self.dynamic_rebalancing(historical_slice, current_vol_btc, current_vol_paxg, row):
                    current_value = self.positions['BTC'] * row['BTC'] + self.positions['PAXG'] * row['PAXG']
                    btc_value_before = self.positions['BTC'] * row['BTC']
                    paxg_value_before = self.positions['PAXG'] * row['PAXG']

                    self.positions['BTC'] = (current_value * self.btc_allocation) / row['BTC']
                    self.positions['PAXG'] = (current_value * self.paxg_allocation) / row['PAXG']
                    btc_value_after = self.positions['BTC'] * row['BTC']
                    paxg_value_after = self.positions['PAXG'] * row['PAXG']
                    fees = (abs(btc_value_after - btc_value_before) + abs(paxg_value_after - paxg_value_before)) * self.transaction_cost
                    transaction_fees.iloc[i] = fees
                    self.last_rebalance = date

            btc_alloc_series.iloc[i] = self.btc_allocation
            paxg_alloc_series.iloc[i] = self.paxg_allocation
            portfolio_value.iloc[i] = self.positions['BTC'] * row['BTC'] + self.positions['PAXG'] * row['PAXG'] - transaction_fees.cumsum().iloc[i]

        bh_value = (self.initial_capital * 0.5 / data['BTC'].iloc[0]) * data['BTC'] + (self.initial_capital * 0.5 / data['PAXG'].iloc[0]) * data['PAXG']
        results = pd.DataFrame({
            'portfolio_value': portfolio_value,
            'buy_hold_value': bh_value,
            'dca_value': dca_value,
            'btc_allocation': btc_alloc_series,
            'paxg_allocation': paxg_alloc_series,
            'transaction_fees': transaction_fees
        }, index=data.index)
        self.logger.info("Backtest completed")
        return results

    def calculate_metrics(self, results: pd.DataFrame, historical_data: pd.DataFrame) -> Dict:
        daily_returns = np.log(results['portfolio_value'] / results['portfolio_value'].shift(1)).dropna()
        bh_daily_returns = np.log(results['buy_hold_value'] / results['buy_hold_value'].shift(1)).dropna()
        dca_daily_returns = np.log(results['dca_value'] / results['dca_value'].shift(1)).dropna()

        years = (results.index[-1] - results.index[0]).days / 365.25

        metrics = {
            'rendement_total_alloy': (results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0] - 1) * 100,
            'rendement_total_buy_hold': (results['buy_hold_value'].iloc[-1] / results['buy_hold_value'].iloc[0] - 1) * 100,
            'rendement_total_dca': (results['dca_value'].iloc[-1] / results['dca_value'].iloc[0] - 1) * 100,
            'rendement_annualise_alloy': ((results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0]) ** (1/years) - 1) * 100,
            'rendement_annualise_buy_hold': ((results['buy_hold_value'].iloc[-1] / results['buy_hold_value'].iloc[0]) ** (1/years) - 1) * 100,
            'rendement_annualise_dca': ((results['dca_value'].iloc[-1] / results['dca_value'].iloc[0]) ** (1/years) - 1) * 100,
            'volatilite_annualisee_alloy': (daily_returns.std() * np.sqrt(252)) * 100,
            'volatilite_annualisee_buy_hold': (bh_daily_returns.std() * np.sqrt(252)) * 100,
            'volatilite_annualisee_dca': (dca_daily_returns.std() * np.sqrt(252)) * 100,
            'ratio_sharpe_alloy': (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252)) if daily_returns.std() != 0 else 0,
            'ratio_sharpe_buy_hold': (bh_daily_returns.mean() * 252) / (bh_daily_returns.std() * np.sqrt(252)) if bh_daily_returns.std() != 0 else 0,
            'ratio_sharpe_dca': (dca_daily_returns.mean() * 252) / (dca_daily_returns.std() * np.sqrt(252)) if dca_daily_returns.std() != 0 else 0,
            'drawdown_maximum_alloy': ((results['portfolio_value'].cummax() - results['portfolio_value']) / results['portfolio_value'].cummax()).max() * 100,
            'drawdown_maximum_buy_hold': ((results['buy_hold_value'].cummax() - results['buy_hold_value']) / results['buy_hold_value'].cummax()).max() * 100,
            'drawdown_maximum_dca': ((results['dca_value'].cummax() - results['dca_value']) / results['dca_value'].cummax()).max() * 100,
            'drawdown_moyen_alloy': ((results['portfolio_value'].cummax() - results['portfolio_value']) / results['portfolio_value'].cummax()).mean() * 100,
            'total_trades': (results['transaction_fees'] > 0).sum(),
            'total_fees': results['transaction_fees'].sum()
        }
        return metrics

    def optimize_parameters(self, data: pd.DataFrame, n_trials: int = 50) -> Dict:
        def objective(trial):
            params = {
                'momentum_window': trial.suggest_int('momentum_window', 10, 60),
                'volatility_window': trial.suggest_int('volatility_window', 20, 100),
                'momentum_threshold_bull': trial.suggest_float('momentum_threshold_bull', 0, 20),
                'momentum_threshold_bear': trial.suggest_float('momentum_threshold_bear', -20, 0),
                'max_btc_allocation': trial.suggest_float('max_btc_allocation', 0.5, 1.0),
                'min_btc_allocation': trial.suggest_float('min_btc_allocation', 0.0, 0.5),
                'rebalance_frequency': trial.suggest_int('rebalance_frequency', 1, 30)
            }
            portfolio = AlloyPortfolioBOME(
                initial_capital=self.initial_capital,
                transaction_cost=self.transaction_cost,
                data=data,
                **params
            )
            results = portfolio.backtest(data)
            metrics = portfolio.calculate_metrics(results, data)
            return metrics['ratio_sharpe_alloy']

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        return study.best_params


## Backtesting

Run a backtest to evaluate the strategy’s performance against Buy & Hold and DCA benchmarks. Adjust parameters interactively and view detailed metrics.

In [ ]:
from ipywidgets import IntSlider, FloatSlider, Button, Output

initial_capital_slider = IntSlider(min=1000, max=1000000, value=10000, step=1000, description='Initial Capital')
momentum_window_slider = IntSlider(min=10, max=60, value=30, step=5, description='Momentum Window')
volatility_window_slider = IntSlider(min=20, max=100, value=60, step=10, description='Volatility Window')
momentum_bull_slider = IntSlider(min=0, max=20, value=10, step=1, description='Bull Threshold')
momentum_bear_slider = IntSlider(min=-20, max=0, value=-5, step=1, description='Bear Threshold')
max_btc_slider = FloatSlider(min=0.5, max=1.0, value=0.9, step=0.05, description='Max BTC Alloc')
min_btc_slider = FloatSlider(min=0.0, max=0.5, value=0.2, step=0.05, description='Min BTC Alloc')
rebalance_freq_slider = IntSlider(min=1, max=30, value=3, step=1, description='Rebalance Freq')
transaction_cost_slider = FloatSlider(min=0.0, max=0.01, value=0.001, step=0.0005, description='Trans Cost')
run_backtest_button = Button(description='Run Backtest')
backtest_output = Output()

def on_run_backtest_clicked(b):
    with backtest_output:
        backtest_output.clear_output()
        if historical_data is None:
            display(HTML("<b style='color:red'>Error: Load data first.</b>"))
            return

        portfolio = AlloyPortfolioBOME(
            initial_capital=initial_capital_slider.value,
            momentum_window=momentum_window_slider.value,
            volatility_window=volatility_window_slider.value,
            momentum_threshold_bull=momentum_bull_slider.value,
            momentum_threshold_bear=momentum_bear_slider.value,
            max_btc_allocation=max_btc_slider.value,
            min_btc_allocation=min_btc_slider.value,
            rebalance_frequency=rebalance_freq_slider.value,
            transaction_cost=transaction_cost_slider.value,
            data=historical_data
        )
        results = portfolio.backtest(historical_data)
        metrics = portfolio.calculate_metrics(results, historical_data)

        # Display metrics in columns
        display(HTML("<h3>Performance Metrics</h3>"))
        html = "<div style='display:flex;justify-content:space-between'>"
        html += "<div style='width:33%;padding:10px'><b>Alloy Strategy</b><br>"
        html += f"Total Return: {metrics['rendement_total_alloy']:.2f}%<br>"
        html += f"Annualized Return: {metrics['rendement_annualise_alloy']:.2f}%<br>"
        html += f"Annualized Volatility: {metrics['volatilite_annualisee_alloy']:.2f}%<br>"
        html += f"Sharpe Ratio: {metrics['ratio_sharpe_alloy']:.2f}<br>"
        html += f"Max Drawdown: {metrics['drawdown_maximum_alloy']:.2f}%<br>"
        html += f"Average Drawdown: {metrics['drawdown_moyen_alloy']:.2f}%<br>"
        html += f"Total Trades: {metrics['total_trades']}<br>"
        html += f"Total Fees: ${metrics['total_fees']:.2f}</div>"

        html += "<div style='width:33%;padding:10px'><b>Buy & Hold</b><br>"
        html += f"Total Return: {metrics['rendement_total_buy_hold']:.2f}%<br>"
        html += f"Annualized Return: {metrics['rendement_annualise_buy_hold']:.2f}%<br>"
        html += f"Annualized Volatility: {metrics['volatilite_annualisee_buy_hold']:.2f}%<br>"
        html += f"Sharpe Ratio: {metrics['ratio_sharpe_buy_hold']:.2f}<br>"
        html += f"Max Drawdown: {metrics['drawdown_maximum_buy_hold']:.2f}%</div>"

        html += "<div style='width:33%;padding:10px'><b>DCA</b><br>"
        html += f"Total Return: {metrics['rendement_total_dca']:.2f}%<br>"
        html += f"Annualized Return: {metrics['rendement_annualise_dca']:.2f}%<br>"
        html += f"Annualized Volatility: {metrics['volatilite_annualisee_dca']:.2f}%<br>"
        html += f"Sharpe Ratio: {metrics['ratio_sharpe_dca']:.2f}<br>"
        html += f"Max Drawdown: {metrics['drawdown_maximum_dca']:.2f}%</div>"
        html += "</div>"
        display(HTML(html))

        # Save results to CSV
        results.to_csv('backtest_results.csv')
        display(HTML("<a href='backtest_results.csv' download>Download Backtest Results as CSV</a>"))

run_backtest_button.on_click(on_run_backtest_clicked)
display(VBox([initial_capital_slider, momentum_window_slider, volatility_window_slider,
              momentum_bull_slider, momentum_bear_slider, max_btc_slider, min_btc_slider,
              rebalance_freq_slider, transaction_cost_slider, run_backtest_button, backtest_output]))

## Results Visualization

Visualize the backtest results with interactive Plotly charts, including portfolio performance, allocations, and drawdowns, compared to benchmarks.

In [ ]:
def plot_results(results: pd.DataFrame, historical_data: pd.DataFrame):
    # Portfolio Performance
    fig = go.Figure()
    normalized_alloy = results['portfolio_value'] / results['portfolio_value'].iloc[0] * 100
    normalized_buy_hold = results['buy_hold_value'] / results['buy_hold_value'].iloc[0] * 100
    normalized_dca = results['dca_value'] / results['dca_value'].iloc[0] * 100
    normalized_btc = historical_data['BTC'] / historical_data['BTC'].iloc[0] * 100
    normalized_paxg = historical_data['PAXG'] / historical_data['PAXG'].iloc[0] * 100

    fig.add_trace(go.Scatter(x=normalized_alloy.index, y=normalized_alloy, mode='lines', name='Alloy Strategy', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=normalized_buy_hold.index, y=normalized_buy_hold, mode='lines', name='Buy & Hold', line=dict(color='orange', dash='dash')))
    fig.add_trace(go.Scatter(x=normalized_dca.index, y=normalized_dca, mode='lines', name='DCA', line=dict(color='green', dash='dot')))
    fig.add_trace(go.Scatter(x=normalized_btc.index, y=normalized_btc, mode='lines', name='BTC', line=dict(color='red', opacity=0.5)))
    fig.add_trace(go.Scatter(x=normalized_paxg.index, y=normalized_paxg, mode='lines', name='PAXG', line=dict(color='purple', opacity=0.5)))
    fig.update_layout(title='Comparative Performance (Base 100)', xaxis_title='Date', yaxis_title='Performance (%)',
                      legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    fig.show()

    # Allocation
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=results.index, y=results['btc_allocation'] * 100, mode='lines', name='BTC Allocation', line=dict(color='orange')))
    fig2.add_trace(go.Scatter(x=results.index, y=results['paxg_allocation'] * 100, mode='lines', name='PAXG Allocation', line=dict(color='purple')))
    fig2.update_layout(title='Dynamic Asset Allocation', xaxis_title='Date', yaxis_title='Allocation (%)',
                       legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), yaxis=dict(range=[0, 100]))
    fig2.show()

    # Drawdown
    fig3 = go.Figure()
    drawdown_alloy = (results['portfolio_value'] - results['portfolio_value'].cummax()) / results['portfolio_value'].cummax() * 100
    drawdown_buy_hold = (results['buy_hold_value'] - results['buy_hold_value'].cummax()) / results['buy_hold_value'].cummax() * 100
    drawdown_dca = (results['dca_value'] - results['dca_value'].cummax()) / results['dca_value'].cummax() * 100
    fig3.add_trace(go.Scatter(x=drawdown_alloy.index, y=drawdown_alloy, mode='lines', name='Alloy Strategy', line=dict(color='blue')))
    fig3.add_trace(go.Scatter(x=drawdown_buy_hold.index, y=drawdown_buy_hold, mode='lines', name='Buy & Hold', line=dict(color='orange', dash='dash')))
    fig3.add_trace(go.Scatter(x=drawdown_dca.index, y=drawdown_dca, mode='lines', name='DCA', line=dict(color='green', dash='dot')))
    fig3.update_layout(title='Drawdowns (%)', xaxis_title='Date', yaxis_title='Drawdown (%)',
                       legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
    fig3.show()

# Note: Run backtest above to generate results, then call plot_results(results, historical_data) manually if needed.

## Parameter Experimentation

Experiment with different parameter combinations to see their impact on performance. Adjust sliders and rerun the backtest.

In [ ]:
@interact(
    initial_capital=IntSlider(min=1000, max=1000000, value=10000, step=1000, description='Initial Capital'),
    momentum_window=IntSlider(min=10, max=60, value=30, step=5, description='Momentum Window'),
    volatility_window=IntSlider(min=20, max=100, value=60, step=10, description='Volatility Window'),
    rebalance_frequency=IntSlider(min=1, max=30, value=3, step=1, description='Rebalance Freq')
)
def experiment_parameters(initial_capital, momentum_window, volatility_window, rebalance_frequency):
    if historical_data is None:
        display(HTML("<b style='color:red'>Error: Load data first.</b>"))
        return

    portfolio = AlloyPortfolioBOME(
        initial_capital=initial_capital,
        momentum_window=momentum_window,
        volatility_window=volatility_window,
        momentum_threshold_bull=10,
        momentum_threshold_bear=-5,
        max_btc_allocation=0.9,
        min_btc_allocation=0.2,
        rebalance_frequency=rebalance_frequency,
        transaction_cost=0.001,
        data=historical_data
    )
    results = portfolio.backtest(historical_data)
    metrics = portfolio.calculate_metrics(results, historical_data)

    display(HTML(f"<b>Total Return:</b> {metrics['rendement_total_alloy']:.2f}% | <b>Sharpe Ratio:</b> {metrics['ratio_sharpe_alloy']:.2f}"))
    plot_results(results, historical_data)

## Trading Signals

Generate a trading signal for a specific date, showing market context and recommended actions, with price and momentum charts.

In [ ]:
signal_date_picker = DatePicker(description='Signal Date', value=datetime.now() - timedelta(days=1))
lookback_days_slider = IntSlider(min=30, max=365, value=180, step=30, description='Lookback Days')
momentum_window_signal_slider = IntSlider(min=10, max=60, value=30, step=5, description='Momentum Window')
momentum_bull_signal_slider = IntSlider(min=0, max=20, value=10, step=1, description='Bull Threshold')
momentum_bear_signal_slider = IntSlider(min=-20, max=0, value=-5, step=1, description='Bear Threshold')
generate_signal_button = Button(description='Generate Signal')
signal_output = Output()

def generate_signal(historical_data, signal_date, lookback_days, momentum_window, momentum_threshold_bull, momentum_threshold_bear):
    start_date = (signal_date - timedelta(days=lookback_days + momentum_window)).strftime('%Y-%m-%d')
    end_date = signal_date.strftime('%Y-%m-%d')
    data = download_data(start_date, end_date)
    if data is None:
        return None

    portfolio = AlloyPortfolioBOME(
        initial_capital=10000,
        momentum_window=momentum_window,
        volatility_window=60,
        momentum_threshold_bull=momentum_threshold_bull,
        momentum_threshold_bear=momentum_threshold_bear,
        max_btc_allocation=0.9,
        min_btc_allocation=0.2,
        rebalance_frequency=3,
        transaction_cost=0.001,
        data=data
    )
    latest_data = data.iloc[-1:]
    vol_btc = portfolio.calculate_volatility(data['BTC']).iloc[-1]
    vol_paxg = portfolio.calculate_volatility(data['PAXG']).iloc[-1]
    btc_momentum = portfolio.calculate_momentum(data['BTC']).iloc[-1]
    context = portfolio.get_market_context(btc_momentum, vol_btc)
    should_rebalance = portfolio.dynamic_rebalancing(data, vol_btc, vol_paxg, latest_data.iloc[0])

    signal = {
        'decision_type': context.split('_')[0],
        'decision_reason': f"BTC momentum ({btc_momentum:.2f}%) indicates {context}",
        'market_context': {
            'btc_price': latest_data['BTC'].iloc[0],
            'btc_30d_perf': ((data['BTC'].iloc[-1] / data['BTC'].iloc[-30]) - 1) * 100 if len(data) >= 30 else 0,
            'btc_momentum': btc_momentum
        },
        'trades': [
            {'type': 'Adjust', 'asset': 'BTC', 'size': 0, 'price': latest_data['BTC'].iloc[0], 'new_allocation': portfolio.btc_allocation * 100},
            {'type': 'Adjust', 'asset': 'PAXG', 'size': 0, 'price': latest_data['PAXG'].iloc[0], 'new_allocation': portfolio.paxg_allocation * 100}
        ] if should_rebalance else []
    }
    return signal, data

def on_generate_signal_clicked(b):
    with signal_output:
        signal_output.clear_output()
        if not signal_date_picker.value:
            display(HTML("<b style='color:red'>Error: Select a valid signal date.</b>"))
            return

        signal_date = signal_date_picker.value
        signal, data = generate_signal(
            historical_data, signal_date, lookback_days_slider.value, momentum_window_signal_slider.value,
            momentum_bull_signal_slider.value, momentum_bear_signal_slider.value
        )

        if signal is None:
            display(HTML("<b style='color:red'>Error: Failed to load data for signal.</b>"))
            return

        # Signal card
        color = 'green' if signal['decision_type'] == 'bullish' else 'red' if signal['decision_type'] == 'bearish' else 'orange'
        display(HTML(f"<h3>Trading Signal for {signal_date.strftime('%Y-%m-%d')}</h3>"
                     f"<div style='border:2px solid {color};padding:10px'>"
                     f"<b style='color:{color}'>{signal['decision_type'].upper()}</b><br>"
                     f"Decision: {signal['decision_reason']}</div>"))

        # Market context
        html = "<div style='display:flex;justify-content:space-between'>"
        html += f"<div style='width:33%'>BTC Price: ${signal['market_context']['btc_price']:,.2f}</div>"
        html += f"<div style='width:33%'>30-Day Perf: {signal['market_context']['btc_30d_perf']:.2f}%</div>"
        html += f"<div style='width:33%'>Momentum: {signal['market_context']['btc_momentum']:.2f}%</div>"
        html += "</div>"
        display(HTML(html))

        # Recommended actions
        if signal['trades']:
            display(HTML("<h4>Recommended Actions</h4>"))
            for trade in signal['trades']:
                display(HTML(f"<b>{trade['type']} {trade['asset']}:</b> New allocation {trade['new_allocation']:.1f}% @ ${trade['price']:,.2f}"))
        else:
            display(HTML("<b>No rebalance required.</b>"))

        # Charts
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index[-90:], y=data['BTC'][-90:], mode='lines', name='BTC Price', line=dict(color='orange')))
        fig.add_trace(go.Scatter(x=data.index[-90:], y=data['PAXG'][-90:], mode='lines', name='PAXG Price', yaxis='y2', line=dict(color='purple')))
        fig.update_layout(title='BTC and PAXG Price (Last 90 Days)', xaxis_title='Date', yaxis_title='BTC Price ($)',
                          yaxis2=dict(title='PAXG Price ($)', overlaying='y', side='right'),
                          legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
        fig.show()

        momentum = (data['BTC'] / data['BTC'].shift(momentum_window_signal_slider.value) - 1) * 100
        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=momentum.index[-90:], y=momentum[-90:], mode='lines', name='BTC Momentum', line=dict(color='blue')))
        fig2.add_shape(type='line', x0=momentum.index[-90], y0=momentum_bull_signal_slider.value, x1=momentum.index[-1], y1=momentum_bull_signal_slider.value,
                       line=dict(color='green', width=2, dash='dash'))
        fig2.add_shape(type='line', x0=momentum.index[-90], y0=momentum_bear_signal_slider.value, x1=momentum.index[-1], y1=momentum_bear_signal_slider.value,
                       line=dict(color='red', width=2, dash='dash'))
        fig2.add_annotation(x=momentum.index[-2], y=momentum_bull_signal_slider.value, text='Bull Threshold', showarrow=False, yshift=10)
        fig2.add_annotation(x=momentum.index[-2], y=momentum_bear_signal_slider.value, text='Bear Threshold', showarrow=False, yshift=-10)
        fig2.update_layout(title='BTC Momentum (Last 90 Days)', xaxis_title='Date', yaxis_title='Momentum (%)')
        fig2.show()

generate_signal_button.on_click(on_generate_signal_clicked)
display(VBox([signal_date_picker, lookback_days_slider, momentum_window_signal_slider,
              momentum_bull_signal_slider, momentum_bear_signal_slider, generate_signal_button, signal_output]))

## Strategy Optimization

Optimize strategy parameters using Optuna to maximize the Sharpe ratio. View the best parameters and top results, with an option to backtest the optimal configuration.

In [ ]:
n_trials_slider = IntSlider(min=10, max=300, value=50, step=10, description='N Trials')
optimize_button = Button(description='Run Optimization')
optimize_output = Output()

def on_optimize_clicked(b):
    with optimize_output:
        optimize_output.clear_output()
        if historical_data is None:
            display(HTML("<b style='color:red'>Error: Load data first.</b>"))
            return

        portfolio = AlloyPortfolioBOME(
            initial_capital=10000, momentum_window=30, volatility_window=60,
            momentum_threshold_bull=10, momentum_threshold_bear=-5,
            max_btc_allocation=0.9, min_btc_allocation=0.2,
            rebalance_frequency=3, transaction_cost=0.001,
            data=historical_data
        )
        best_params = portfolio.optimize_parameters(historical_data, n_trials=n_trials_slider.value)

        # Run backtest with best parameters
        portfolio_opt = AlloyPortfolioBOME(
            initial_capital=10000, transaction_cost=0.001, data=historical_data, **best_params
        )
        results = portfolio_opt.backtest(historical_data)
        metrics = portfolio_opt.calculate_metrics(results, historical_data)

        # Display best parameters
        display(HTML("<h3>Best Parameters</h3>"))
        html = "<div style='display:flex;justify-content:space-between'>"
        html += f"<div style='width:25%'>Momentum Window: {best_params['momentum_window']} days</div>"
        html += f"<div style='width:25%'>Volatility Window: {best_params['volatility_window']} days</div>"
        html += f"<div style='width:25%'>Rebalance Freq: {best_params['rebalance_frequency']} days</div>"
        html += f"<div style='width:25%'>Max BTC Alloc: {best_params['max_btc_allocation']*100:.0f}%</div>"
        html += "</div><div style='display:flex;justify-content:space-between;margin-top:10px'>"
        html += f"<div style='width:25%'>Min BTC Alloc: {best_params['min_btc_allocation']*100:.0f}%</div>"
        html += f"<div style='width:25%'>Bull Threshold: {best_params['momentum_threshold_bull']}%</div>"
        html += f"<div style='width:25%'>Bear Threshold: {best_params['momentum_threshold_bear']}%</div>"
        html += "</div>"
        display(HTML(html))

        # Display metrics
        display(HTML("<h3>Performance with Best Parameters</h3>"))
        html = "<div style='display:flex;justify-content:space-between'>"
        html += f"<div style='width:25%'>Total Return: {metrics['rendement_total_alloy']:.2f}%</div>"
        html += f"<div style='width:25%'>Annualized Return: {metrics['rendement_annualise_alloy']:.2f}%</div>"
        html += f"<div style='width:25%'>Sharpe Ratio: {metrics['ratio_sharpe_alloy']:.2f}</div>"
        html += f"<div style='width:25%'>Max Drawdown: {metrics['drawdown_maximum_alloy']:.2f}%</div>"
        html += "</div>"
        display(HTML(html))

        # Save parameters to JSON
        import json
        with open('optimal_params.json', 'w') as f:
            json.dump(best_params, f, indent=4)
        display(HTML("<a href='optimal_params.json' download>Download Optimal Parameters as JSON</a>"))

        plot_results(results, historical_data)

optimize_button.on_click(on_optimize_clicked)
display(VBox([n_trials_slider, optimize_button, optimize_output]))

## Next Steps

This notebook replicates the core functionality of the Alloy BTC-PAXG Trading Assistant. To explore further:

- **Run the Full Application**: Clone the repository and run `app.py` for a Streamlit interface:
  ```bash
  git clone https://github.com/yourusername/alloy-btc-paxg-trading
  pip install -r requirements.txt
  streamlit run app.py
  ```
- **Live Trading**: Integrate with an exchange API (e.g., Binance). See [xAI API documentation](https://x.ai/api).
- **Contribute**: Share feedback or contribute on GitHub.
- **Community**: Discuss on Reddit: [r/PAXG](https://www.reddit.com/r/PAXG/).

## Troubleshooting
- **Data Errors**: If `yfinance` fails, try a different date range or check your internet connection.
- **Widget Issues**: Ensure `ipywidgets` is enabled (`jupyter nbextension enable --py widgetsnbextension`).
- **Performance**: Optimization with many trials may be slow; reduce `n_trials` for faster results.